In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')


c:\Users\gaomi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [8]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [22]:
lis=[]
for i in tradable_id:
    dir_path=f"individual_book_train/stock_{3}.csv"
    data=pd.read_csv(dir_path)
    lis.append(data.time_id.nunique())

In [24]:
np.array(lis)

array([3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830, 3830,
       3830])

In [9]:
def feature_generator(stock_id,last_min=5,forecast_period=60):
    """_summary_

    Args:
        data (df): dataframe for each stock_id from preprocessing
        stock_id (int): stock_id
        last_min (int, optional): represent how many minutes data I want to summarize Defaults to 5.
        forecast_period (int, optional): forecast how many seconds. Defaults to 60.
    """
    def preprocessor_book(stock_id):
        
        book_dtypes = {
            'stock_id':np.uint16, 
            'time_id': np.uint16,
            'seconds_in_bucket': np.uint16,
            'bid_price1': np.float32,
            'ask_price1': np.float32,
            'bid_price2': np.float32,
            'ask_price2': np.float32,
            'bid_size1': np.uint32,
            'ask_size1': np.uint32,
            'bid_size2': np.uint32,
            'ask_size2': np.uint32,
        }

        dir_path=f"individual_book_train/stock_{stock_id}.csv"
        data=pd.read_csv(dir_path)
        for column, dtype in book_dtypes.items():
            data[column] = data[column].astype(dtype)
        

        return data
    def difference(x):
        return abs(np.max(x) - np.min(x))
    
    data=preprocessor_book(stock_id=stock_id)
    data['wap1'] =(data['bid_price1'] * data['ask_size1'] + data['ask_price1'] * data['bid_size1'])/(data['bid_size1'] + data['ask_size1'])
    data['log_return'] = data.groupby('time_id')['wap1'].transform(log_return)
    data['wap2'] = (data['bid_price2'] * data['ask_size2'] + data['ask_price2'] * data['bid_size2'])/(data['bid_size2'] + data['ask_size2'])
    data['log_return2'] = data.groupby('time_id')['wap2'].transform(log_return)

    data['wap_balance'] = abs(data['wap1'] - data['wap2'])

    data['price_spread'] = (data['ask_price1'] - data['bid_price1']) / ((data['ask_price1'] + data['bid_price1'])/2)
    data['bid_spread'] = data['bid_price1'] - data['bid_price2']
    data['ask_spread'] = data['ask_price1'] - data['ask_price2']
    data['total_volume'] = (data['ask_size1'] + data['ask_size2']) + (data['bid_size1'] + data['bid_size2'])
    data['volume_imbalance'] = abs((data['ask_size1'] + data['ask_size2']) - (data['bid_size1'] + data['bid_size2']))
    data_test=data[data["seconds_in_bucket"]>=600-forecast_period]
    data_train=data[data["seconds_in_bucket"]<600-forecast_period]
    #dict for aggregate
    create_feature_dict = {
        'log_return':[realized_volatility],
        'log_return2':[realized_volatility],
        'wap_balance':[np.mean],
        'price_spread':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'volume_imbalance':[np.mean],
        'total_volume':[np.mean,difference],
        'wap1':[np.std,np.mean,difference],
        'wap2':[np.std,np.mean,difference],
            }

    #####groupby / all seconds
    df_feature = pd.DataFrame(data_train.groupby(['time_id'],sort=False).agg(create_feature_dict)).reset_index()
    last_min=5
    df_feature.columns = ['_'.join(col).strip() for col in df_feature.columns.values]
    df_feature["missing_values"]=pd.DataFrame(data_train.groupby(['time_id'],sort=False).apply(lambda x: 540-len(x))).reset_index().iloc[:,1]
    for minute in range(1,last_min):
        second = 540 - minute*60 

        df_feature_sec = pd.DataFrame(data_train.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
        
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)

    df_feature["stock_id"]=1
    create_target_dict = {
        'log_return':[realized_volatility],
            }
    target = pd.DataFrame(data_test.groupby(['time_id'],sort=False).agg(create_target_dict)).reset_index()
    target.columns = ['_'.join(col).strip() for col in target.columns.values]
    df_feature = pd.merge(df_feature,target,how='left',left_on='time_id_',right_on=f'time_id_')
    df_feature.rename(columns={'log_return_realized_volatility_y': 'target'}, inplace=True)
    df_feature["stock_id"]=stock_id
    return df_feature

In [103]:
df_feat=feature_generator(1)
df_feat

,time_id_,log_return_realized_volatility_x,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,total_volume_difference,wap1_std,wap1_mean,wap1_difference,wap2_std,wap2_mean,wap2_difference,missing_values,log_return_realized_volatility_480,log_return2_realized_volatility_480,wap_balance_mean_480,price_spread_mean_480,bid_spread_mean_480,ask_spread_mean_480,volume_imbalance_mean_480,total_volume_mean_480,total_volume_difference_480,wap1_std_480,wap1_mean_480,wap1_difference_480,wap2_std_480,wap2_mean_480,wap2_difference_480,log_return_realized_volatility_420,log_return2_realized_volatility_420,wap_balance_mean_420,price_spread_mean_420,bid_spread_mean_420,ask_spread_mean_420,volume_imbalance_mean_420,total_volume_mean_420,total_volume_difference_420,wap1_std_420,wap1_mean_420,wap1_difference_420,wap2_std_420,wap2_mean_420,wap2_difference_420,log_return_realized_volatility_360,log_return2_realized_volatility_360,wap_balance_mean_360,price_spread_mean_360,bid_spread_mean_360,ask_spread_mean_360,volume_imbalance_mean_360,total_volume_mean_360,total_volume_difference_360,wap1_std_360,wap1_mean_360,wap1_difference_360,wap2_std_360,wap2_mean_360,wap2_difference_360,log_return_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,volume_imbalance_mean_300,total_volume_mean_300,total_volume_difference_300,wap1_std_300,wap1_mean_300,wap1_difference_300,wap2_std_300,wap2_mean_300,wap2_difference_300,stock_id,target
0,5,0.006001,0.007367,0.000271,0.000675,0.000114,-0.000105,1.651911e+09,293.986538,1983,0.001203,1.003659,0.005658,0.001242,1.003692,0.005632,20,0.001848,0.002146,0.000268,0.000664,0.000079,-0.000086,1.733057e+09,289.280702,570,0.000476,1.003780,0.001926,0.000585,1.003918,0.001828,0.002545,0.003076,0.000306,0.000682,0.000091,-0.000095,1.276882e+09,274.954955,632,0.000449,1.003916,0.001939,0.000537,1.004087,0.001933,0.003185,0.004025,0.000276,0.000660,0.000096,-0.000105,1.035845e+09,275.588235,663,0.000569,1.003660,0.002332,0.000660,1.003769,0.002361,0.003681,0.004585,0.000266,0.000636,0.000101,-0.000102,1.050298e+09,256.419214,663,0.000568,1.003730,0.002332,0.000636,1.003814,0.002361,1,0.001727
1,11,0.002148,0.002916,0.000230,0.000481,0.000117,-0.000107,1.931465e+09,320.795858,830,0.000370,1.001762,0.001784,0.000403,1.001766,0.001941,202,0.000791,0.000697,0.000163,0.000538,0.000161,-0.000098,3.789678e+08,255.088235,369,0.000167,1.001775,0.000462,0.000166,1.001659,0.000653,0.001297,0.001160,0.000225,0.000495,0.000139,-0.000098,1.541783e+09,324.397436,755,0.000324,1.001993,0.001187,0.000411,1.002019,0.001334,0.001374,0.001672,0.000225,0.000458,0.000142,-0.000100,1.296594e+09,306.971698,755,0.000318,1.001945,0.001261,0.000409,1.001984,0.001503,0.001560,0.001912,0.000217,0.000446,0.000140,-0.000097,1.501737e+09,319.650350,755,0.000317,1.001869,0.001588,0.000412,1.001867,0.001680,1,0.000837
2,16,0.002356,0.002840,0.000087,0.000293,0.000117,-0.000118,2.078654e+09,605.301282,1148,0.000893,1.000972,0.003273,0.000900,1.000947,0.003458,228,0.000791,0.000920,0.000082,0.000242,0.000113,-0.000131,1.625123e+09,661.594595,513,0.000264,1.002141,0.001068,0.000320,1.002131,0.001329,0.001161,0.001398,0.000082,0.000245,0.000119,-0.000126,2.312675e+09,619.230769,579,0.000242,1.002051,0.001068,0.000274,1.002033,0.001329,0.001595,0.001828,0.000086,0.000257,0.000124,-0.000118,2.465629e+09,605.805556,736,0.000298,1.001964,0.001429,0.000332,1.001938,0.001698,0.001871,0.002152,0.000083,0.000270,0.000119,-0.000118,2.176117e+09,635.013333,1111,0.000480,1.001735,0.002074,0.000505,1.001705,0.002325,1,0.000886
3,31,0.003376,0.004092,0.000341,0.000796,0.000188,-0.000154,1.921433e+09,428.736842,1272,0.000609,0.997548,0.002284,0.000639,0.997471,0.002682,388,0.000292,0.000716,0.000317,0.000697,0.000135,-0.000135,2.555714e+02,465.285714,393,0.000003,0.996381,0.000007,0.000238,0.996662,0.0005

In [10]:
from tqdm import tqdm
non_tradable=[]
for i in tqdm(range(0,127)):
    try:
        dir_path=f"individual_book_train/stock_{i}.csv"
        data=pd.read_csv(dir_path)
    except:
        non_tradable.append(i)

tradable_id=[i for i in range(0,127) if i not in non_tradable]
tradable_id

100%|██████████| 127/127 [01:22<00:00,  1.53it/s]


[0,
 1,
 2,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 46,
 47,
 48,
 50,
 51,
 52,
 53,
 55,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 66,
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 118,
 119,
 120,
 122,
 123,
 124,
 125,
 126]

In [11]:
len(tradable_id)

111

In [12]:
def preprocessor(list_stock_ids):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
    
            
        df_tmp = feature_generator(stock_id=stock_id)
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

In [14]:
final_df=preprocessor(tradable_id)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 111 out of 111 | elapsed:  1.9min finished


In [15]:
final_df

,time_id_,log_return_realized_volatility_x,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,total_volume_difference,wap1_std,wap1_mean,wap1_difference,wap2_std,wap2_mean,wap2_difference,missing_values,log_return_realized_volatility_480,log_return2_realized_volatility_480,wap_balance_mean_480,price_spread_mean_480,bid_spread_mean_480,ask_spread_mean_480,volume_imbalance_mean_480,total_volume_mean_480,total_volume_difference_480,wap1_std_480,wap1_mean_480,wap1_difference_480,wap2_std_480,wap2_mean_480,wap2_difference_480,log_return_realized_volatility_420,log_return2_realized_volatility_420,wap_balance_mean_420,price_spread_mean_420,bid_spread_mean_420,ask_spread_mean_420,volume_imbalance_mean_420,total_volume_mean_420,total_volume_difference_420,wap1_std_420,wap1_mean_420,wap1_difference_420,wap2_std_420,wap2_mean_420,wap2_difference_420,log_return_realized_volatility_360,log_return2_realized_volatility_360,wap_balance_mean_360,price_spread_mean_360,bid_spread_mean_360,ask_spread_mean_360,volume_imbalance_mean_360,total_volume_mean_360,total_volume_difference_360,wap1_std_360,wap1_mean_360,wap1_difference_360,wap2_std_360,wap2_mean_360,wap2_difference_360,log_return_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,volume_imbalance_mean_300,total_volume_mean_300,total_volume_difference_300,wap1_std_300,wap1_mean_300,wap1_difference_300,wap2_std_300,wap2_mean_300,wap2_difference_300,stock_id,target
0,5,0.004323,0.006440,0.000392,0.000864,0.000173,-0.000153,2.351259e+09,334.861314,728,0.000671,1.003792,0.003486,0.000767,1.003724,0.003735,266,0.000774,0.001268,0.000302,0.000843,0.000293,-0.000197,1.908874e+09,276.333333,327.0,0.000173,1.003786,0.000429,0.000396,1.003632,0.000871,0.001344,0.003385,0.000355,0.000774,0.000237,-0.000226,2.675553e+09,307.606557,525,0.000184,1.003836,0.000858,0.000474,1.003744,0.001956,0.002119,0.003819,0.000407,0.000763,0.000246,-0.000199,2.796723e+09,282.883721,525,0.000264,1.003915,0.001115,0.000484,1.003829,0.002061,0.002678,0.004019,0.000379,0.000844,0.000228,-0.000170,2.437684e+09,315.774775,663,0.000270,1.003924,0.001341,0.000449,1.003825,0.002061,0,0.001246
1,11,0.000918,0.002110,0.000194,0.000397,0.000106,-0.000136,2.726580e+09,402.084270,830,0.000199,1.000179,0.001040,0.000257,1.000170,0.001491,362,0.000442,0.001270,0.000202,0.000305,0.000107,-0.000186,2.684355e+09,347.166667,339.0,0.000148,1.000390,0.000582,0.000333,1.000393,0.001366,0.000497,0.001366,0.000239,0.000345,0.000088,-0.000139,2.938662e+09,441.447368,499,0.000125,1.000390,0.000582,0.000301,1.000293,0.001366,0.000585,0.001493,0.000227,0.000375,0.000091,-0.000126,2.863311e+09,513.666667,693,0.000116,1.000340,0.000595,0.000255,1.000294,0.001366,0.000596,0.001534,0.000212,0.000349,0.000100,-0.000122,2.909494e+09,483.881720,830,0.000113,1.000320,0.000595,0.000234,1.000311,0.001366,0,0.000779
2,16,0.002346,0.004511,0.000326,0.000732,0.000189,-0.000195,2.386093e+09,415.816667,626,0.000756,0.999636,0.003423,0.000750,0.999773,0.003979,360,0.000704,0.001915,0.000284,0.000535,0.000206,-0.000168,4.294969e+08,461.800000,394.0,0.000241,0.998176,0.001037,0.000423,0.998352,0.002153,0.001112,0.002489,0.000368,0.000616,0.000139,-0.000187,7.708917e+08,463.820513,476,0.000439,0.998467,0.001629,0.000698,0.998757,0.003323,0.001173,0.002728,0.000416,0.000686,0.000115,-0.000215,1.134520e+09,472.943396,476,0.000564,0.998710,0.002140,0.000807,0.999064,0.003414,0.001254,0.002741,0.000429,0.000704,0.000112,-0.000247,1.288490e+09,458.816667,509,0.000667,0.998853,0.002736,0.000770,0.999114,0.003414,0,0.000323
3,31,0.002399,0.003378,0.000391,0.000845,0.000192,-0.000112,1.900428e+09,428.823009,737,0.000720,0.998899,0.002397,0.000637,0.998677,0.002609,427,0.000326,0.000319,0.000235,0.001012,0.000085,-0.000054,3.579139e+09,519.833333,247.0,0.000177,0.998464,0.000476,0.000081,0.998699,0

In [16]:
final_df.to_csv("merge_regression_data.csv")

In [ ]:
11

In [26]:
for id in final_df.stock_id.unique():
    print(final_df[final_df["stock_id"]==id].shape)

(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3829, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3815, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3829, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3820, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3830, 79)
(3829, 79)
(3830, 79)
(3830, 79)
(3830, 79)

In [3]:
merge_data=pd.read_pickle('merge_regression_data.pkl')
features=["log_return_realized_volatility","log_return2_realized_volatility","wap_balance_mean","price_spread_mean","bid_spread_mean","ask_spread_mean","volume_imbalance_mean","total_volume_mean","wap1_mean","wap2_mean"]
merge_data

,Unnamed: 0,minute_id_,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap1_mean,wap2_mean,stock_id,time_id,target
0,0,5-0,0.001649,0.002227,0.000275,0.000873,0.000126,-0.000163,166.880000,274.160000,1.002064,1.001879,0,5,0.001151
1,1,5-1,0.001151,0.001364,0.000278,0.000874,0.000086,-0.000105,168.461538,317.461538,1.003939,1.003824,0,5,0.001994
2,2,5-2,0.001994,0.002603,0.000532,0.000753,0.000144,-0.000182,116.700000,360.350000,1.003939,1.004309,0,5,0.001525
3,3,5-3,0.001525,0.002774,0.000444,0.000936,0.000165,-0.000095,79.408163,400.142857,1.004152,1.003870,0,5,0.001083
4,4,5-4,0.001083,0.002009,0.000353,0.000976,0.000124,-0.000184,198.347826,329.217391,1.003840,1.003802,0,5,0.001637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4289072,38294,32767-5,0.000865,0.000873,0.000209,0.000394,0.000129,-0.000146,133.500000,339.055556,1.000523,1.000461,126,32767,0.000381
4289073,38295,32767-6,0.000381,0.001071,0.000237,0.000368,0.000129,-0.000287,93.250000,369.750000,1.000610,1.000789,126,32767,0.000736
4289074,38296,32767-7,0.000736,0.001672,0.000254,0.000415,0.000121,-0.000208,195.076923,498.846154,1.000687,1.000751,126,32767,0.000372
4289075,38297,32767-8,0.000372,0.000690,0.000289,0.000475,0.000146,-0.000119,263.538462,935.692308,1.000149,1.000438,126,32767,0.000654


In [5]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
merge_data.dropna(subset=["target"]+features,inplace=True)

X=merge_data[features]
y=merge_data["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = LinearRegression()

# Fit the model to the data
model.fit(X_train, y_train)

# Predict
pred=model.predict(X_test)

def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculate the Mean Absolute Percentage Error (MAPE).

    Parameters:
        y_true (array-like): Array of true values.
        y_pred (array-like): Array of predicted values.

    Returns:
        float: MAPE value.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mean_absolute_percentage_error(pred,y_test)


31.2796771256678

In [38]:
features

['log_return_realized_volatility',
 'log_return2_realized_volatility',
 'wap_balance_mean',
 'price_spread_mean',
 'bid_spread_mean',
 'ask_spread_mean',
 'volume_imbalance_mean',
 'total_volume_mean',
 'wap1_mean',
 'wap2_mean',
 'log_return_realized_volatility_lag1',
 'log_return2_realized_volatility_lag1',
 'log_return_realized_volatility_lag2',
 'log_return2_realized_volatility_lag2',
 'log_return_realized_volatility_lag3',
 'log_return2_realized_volatility_lag3',
 'log_return_realized_volatility_lag4',
 'log_return2_realized_volatility_lag4',
 'log_return_realized_volatility_lag5',
 'log_return2_realized_volatility_lag5',
 'log_return_realized_volatility_lag6',
 'log_return2_realized_volatility_lag6']

In [37]:
import xgboost as xgb
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define parameters for XGBoost
params = {
    'objective': 'reg:squarederror',  # Regression task
    'eval_metric': 'mape'              # Evaluation metric
}

# Train the XGBoost model
num_rounds = 1000  # Number of boosting rounds
model = xgb.train(params, dtrain, num_rounds)

# Make predictions
predictions = model.predict(dtest)

# Evaluate the model
rmse = mean_absolute_percentage_error(y_test, predictions)
print("RMSE:", rmse)

RMSE: 65.24105304108264


AttributeError: module 'xgboost' has no attribute 'get_booster'